# 🚦 YOLO Detection Server V2
### HLS Stream + WebSocket Results

**Flow:**
1. Pull frames from HLS stream (Node Media Server)
2. Run YOLO detection
3. Send results via WebSocket to backend

In [ ]:
# ═══════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION - EDIT THIS!
# ═══════════════════════════════════════════════════════════

# HLS stream URL from your Node Media Server
HLS_STREAM_URL = 'http://YOUR_SERVER:8000/live/security_gate/index.m3u8'

# WebSocket URL for sending results (expose via localhost.run)
BACKEND_WS_URL = 'ws://YOUR_SERVER:3002'

# Camera ID (should match your camera config)
CAMERA_ID = 'security_gate'

# Detection settings
TRACK_LINE_Y = 50  # % from top
CONFIDENCE = 0.5
FPS_LIMIT = 5  # Max FPS for detection

print(f"📺 HLS: {HLS_STREAM_URL}")
print(f"📡 WS: {BACKEND_WS_URL}")

In [ ]:
# Cell 2: Install Dependencies
!pip install ultralytics opencv-python-headless websocket-client --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import os
if not os.path.exists('yolov5'):
    !git clone --depth 1 https://github.com/ultralytics/yolov5.git 2>/dev/null

# Patch yolov5
for f, old, new in [
    ('yolov5/utils/plots.py', 'import seaborn as sn', '# seaborn'),
    ('yolov5/utils/plots.py', 'from scipy.ndimage.filters import gaussian_filter1d', '# scipy'),
    ('yolov5/models/yolo.py', 'from utils.plots import feature_visualization', '# plots')
]:
    try:
        with open(f, 'r') as x: c = x.read()
        with open(f, 'w') as x: x.write(c.replace(old, new))
    except: pass

print('✅ Dependencies installed')

In [ ]:
# Cell 3: Load Models
import torch, sys, os
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

vehicle_model = traffic_light_model = lp_detector_model = None

# Vehicle
try:
    vehicle_model = YOLO('yolo11m.pt').to(device)
    print('✅ Vehicle')
except Exception as e: print(f'⚠️ Vehicle: {e}')

# Traffic Light
try:
    p = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-dentinhieu-best-new.pt'
    if os.path.exists(p):
        traffic_light_model = YOLO(p).to(device)
        print('✅ Traffic Light')
except Exception as e: print(f'⚠️ TL: {e}')

# License Plate
try:
    lp_det = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_detector.pt'
    if os.path.exists(lp_det) and 'yolov5' not in sys.path:
        sys.path.insert(0, os.path.abspath('yolov5'))
        lp_detector_model = torch.load(lp_det, map_location=device, weights_only=False)['model'].float().eval()
        if device == 'cuda': lp_detector_model = lp_detector_model.cuda()
        print('✅ LP')
except Exception as e: print(f'⚠️ LP: {e}')

print(f"\n📊 V:{'✅' if vehicle_model else '❌'} TL:{'✅' if traffic_light_model else '❌'} LP:{'✅' if lp_detector_model else '❌'}")

In [ ]:
# Cell 4: Detection Functions
import cv2
import numpy as np
import time
import json
import threading
from datetime import datetime

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
camera_trackers = {}

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def detect_frame(frame, camera_id):
    """Run detection on a frame, return results dict"""
    h, w = frame.shape[:2]
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h}
    }
    
    # Init tracker
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = {
            'tracks': {}, 'counted': {},
            'counts_up': {v:0 for v in VEHICLE_CLASSES},
            'counts_down': {v:0 for v in VEHICLE_CLASSES},
            'total_up': 0, 'total_down': 0
        }
    tr = camera_trackers[camera_id]
    
    # Vehicle Detection
    if vehicle_model:
        t0 = time.time()
        dets, tracks, vcounts, crossings = [], {}, {v:0 for v in VEHICLE_CLASSES}, []
        ly = int(h * TRACK_LINE_Y / 100)
        
        for r in vehicle_model.track(frame, persist=True, verbose=False):
            for b in r.boxes:
                cls = vehicle_model.names[int(b.cls[0])]
                if cls not in VEHICLE_CLASSES or float(b.conf[0]) < CONFIDENCE: continue
                x1,y1,x2,y2 = map(int, b.xyxy[0])
                cx, cy = (x1+x2)//2, (y1+y2)//2
                det = {'class':cls, 'type':'vehicle', 'confidence':float(b.conf[0]),
                       'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}}
                if hasattr(b,'id') and b.id is not None:
                    tid = int(b.id[0]); det['id'] = tid
                    tracks[tid] = {'pos': (cx,cy), 'time': result['created_at'], 'class': cls}
                dets.append(det); vcounts[cls] += 1
        
        # Counting
        for tid, info in tracks.items():
            if tid not in tr['tracks']: tr['tracks'][tid] = []
            if tr['tracks'][tid]:
                py, cy = tr['tracks'][tid][-1]['pos'][1], info['pos'][1]
                d = 1 if py <= ly < cy else (-1 if py >= ly > cy else 0)
                if d and f"{tid}_{d}" not in tr['counted']:
                    tr['counted'][f"{tid}_{d}"] = True
                    if d == 1: tr['counts_down'][info['class']] += 1; tr['total_down'] += 1
                    else: tr['counts_up'][info['class']] += 1; tr['total_up'] += 1
                    crossings.append({'id': tid, 'direction': d})
            tr['tracks'][tid].append({'pos': info['pos'], 'time': info['time'], 'class': info['class']})
            tr['tracks'][tid] = tr['tracks'][tid][-30:]
        
        result['vehicle'] = {
            'detections': dets,
            'inference_time': (time.time()-t0)*1000,
            'vehicle_count': {
                'total_up': tr['total_up'], 'total_down': tr['total_down'],
                'by_type_up': tr['counts_up'].copy(), 'by_type_down': tr['counts_down'].copy(),
                'current': vcounts
            },
            'new_crossings': crossings
        }
    
    # Traffic Light
    if traffic_light_model:
        t0 = time.time(); tl_dets = []; status = None; mx = 0
        for r in traffic_light_model(frame, verbose=False):
            for b in r.boxes:
                cf = float(b.conf[0])
                if cf < 0.4: continue
                x1,y1,x2,y2 = map(int, b.xyxy[0])
                cn = traffic_light_model.names[int(b.cls[0])]
                tl_dets.append({'class':cn,'confidence':cf,
                               'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
                if cf > mx: mx, status = cf, cn
        result['traffic_light'] = {
            'detections': tl_dets,
            'traffic_status': status,
            'inference_time': (time.time()-t0)*1000
        }
    
    return result

print('✅ Detection functions ready')

In [ ]:
# Cell 5: WebSocket Client + HLS Stream Reader (RUNS FOREVER)
import websocket
import cv2
import json
import time
import threading

ws = None
connected = False
frame_count = 0
detect_count = 0

def on_open(ws_client):
    global connected
    connected = True
    log('🔗 Connected to backend')

def on_close(ws_client, code, reason):
    global connected
    connected = False
    log(f'❌ Disconnected: {code} {reason}')

def on_error(ws_client, error):
    log(f'⚠️ WS Error: {error}')

def on_message(ws_client, message):
    log(f'📩 Message: {message[:100]}')

def send_result(result):
    global detect_count
    if ws and connected:
        try:
            ws.send(json.dumps({'type': 'detection_result', 'data': result}))
            detect_count += 1
        except Exception as e:
            log(f'❌ Send error: {e}')

def start_ws_client():
    global ws
    while True:
        try:
            ws = websocket.WebSocketApp(
                BACKEND_WS_URL,
                on_open=on_open,
                on_close=on_close,
                on_error=on_error,
                on_message=on_message
            )
            ws.run_forever()
        except Exception as e:
            log(f'❌ WS Connection failed: {e}')
        time.sleep(5)  # Reconnect delay

# Start WebSocket client in background
threading.Thread(target=start_ws_client, daemon=True).start()
time.sleep(2)

# Main loop: Read HLS stream and detect
print('═' * 60)
print(f'🎥 Connecting to HLS stream: {HLS_STREAM_URL}')
print('═' * 60)

frame_interval = 1.0 / FPS_LIMIT
last_detect_time = 0

while True:
    try:
        cap = cv2.VideoCapture(HLS_STREAM_URL)
        if not cap.isOpened():
            log('❌ Cannot open stream, retrying in 5s...')
            time.sleep(5)
            continue
        
        log('✅ Stream connected')
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                log('⚠️ Frame read failed, reconnecting...')
                break
            
            frame_count += 1
            now = time.time()
            
            # Rate limit detection
            if now - last_detect_time >= frame_interval:
                last_detect_time = now
                
                # Run detection
                result = detect_frame(frame, CAMERA_ID)
                
                # Send to backend
                send_result(result)
                
                # Log progress every 100 detections
                if detect_count % 100 == 0:
                    log(f'📊 Frames: {frame_count} | Detections: {detect_count} | Connected: {connected}')
        
        cap.release()
        
    except Exception as e:
        log(f'❌ Error: {e}')
        time.sleep(5)